In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24566173
paper_name = 'bleackley_anderson_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/full list NaD1 resistance.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5153 x 10


In [7]:
original_data.head()

,plate,row,column,z score,difference,pgi,gene,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,15,B,6,-4.589586,0.0033,-550.000000,YHR060W,NaN,NaN,100.660630
1,70,G,5,-2.579425,0.0062,-442.857143,YOR331C,NaN,NaN,10.968362
2,47,A,2,-4.464301,0.0055,-275.000000,YBR127C,NaN,NaN,NaN
3,3,A,5,-4.511339,0.0061,-132.608696,YLR122C,NaN,NaN,NaN
4,12,G,10,-4.650589,0.0052,-120.930233,YEL051W,NaN,NaN,89.692268


In [8]:
original_data['orf'] = original_data['gene'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             plate row  column   z score  difference    pgi gene  Unnamed: 7  \
index_input                                                                    
5150            40   D       4 -4.415866      0.0066  412.5  NaN         NaN   

             Unnamed: 8  Unnamed: 9  orf  
index_input                               
5150                NaN         NaN  NAN  


In [13]:
original_data = original_data.loc[t,:]

In [14]:
original_data['data'] = -original_data['pgi'].astype(float)

In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(4942, 1)

# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [757]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,757
data_type,value
orf,
YAL002W,-97.607935
YAL004W,-101.981051
YAL005C,-101.759657
YAL007C,-101.283582
YAL008W,-99.894589


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,757
,data_type,value
gene_id,orf,
2,YAL002W,-97.607935
1863,YAL004W,-101.981051
4,YAL005C,-101.759657
5,YAL007C,-101.283582
6,YAL008W,-99.894589


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id              757          
data_type             value    valuez
gene_id orf                          
2       YAL002W  -97.607935  0.441504
1863    YAL004W -101.981051  0.231873
4       YAL005C -101.759657  0.242486
5       YAL007C -101.283582  0.265307
6       YAL008W  -99.894589  0.331890

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24566173...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.16s/it]

Updating the data_modified_on field...
